# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [27]:
import pandas as pd
document = ET.parse( './data/mondial_database.xml' )


In [48]:
#traverse the tree, recording all infant_mortality rates for countries 
inf_mort= {}
for child in document.getroot():
    n = child.find('name')
    i = child.find('infant_mortality')
    if (n is not None) & (i is not None):
        inf_mort[n.text] = float(i.text)

#display the 10 lowest
pd.Series(inf_mort).sort_values().head(10)

Monaco            1.81
Japan             2.13
Norway            2.48
Bermuda           2.48
Singapore         2.53
Sweden            2.60
Czech Republic    2.63
Hong Kong         2.73
Macao             3.13
Iceland           3.15
dtype: float64

In [95]:
#traverse the XML tree, getting yearly population data for each city in each country
city_pop= []
for element in document.iterfind('country'):
    country_name = element.find('name').text
    for subelement in element.getiterator('city'):
        city = subelement.find('name').text
        for pop in subelement.getiterator('population'):
            row = {}
            row['city'] = city
            row['country'] = country_name
            row['population'] = float(pop.text)
            row['year'] = int(pop.attrib['year'])
            city_pop.append(row)

pop_table = pd.DataFrame(city_pop)

#return the latest yearly population for each city by dropping duplicates and keeping only the latest year.  return the top 10 most populous
pop_table.sort_values('year').drop_duplicates(['city', 'country'], keep='last').sort_values('population', ascending=False).head(10)

,city,country,population,year
3750,Shanghai,China,22315474.0,2010
2607,Istanbul,Turkey,13710512.0,2012
4303,Mumbai,India,12442373.0,2011
1546,Moskva,Russia,11979529.0,2013
3746,Beijing,China,11716620.0,2010
8208,São Paulo,Brazil,11152344.0,2010
3754,Tianjin,China,11090314.0,2010
3364,Guangzhou,China,11071424.0,2010
4399,Delhi,India,11034555.0,2011
3371,Shenzhen,China,10358381.0,2010


In [98]:
#in order to calculate populations of ethnic groups (which are only given as percentages of the total population)
#we need each country's total population
cty_pop =[]
for element in document.iterfind('country'):
    country_name = element.find('name').text
    for pop in element.getiterator('population'):
        row={}
        row['year'] = int(pop.attrib['year'])
        row['population'] = float(pop.text)
        row['country'] = country_name
        cty_pop.append(row)

#since the population figures are yearly, we have to drop all but the most recent as in the previous answer
pop_table = pd.DataFrame(cty_pop)
cur_pop = pop_table.sort_values('year').drop_duplicates(['country'], keep='last')
cur_pop

,country,population,year
12687,Pitcairn,66.0,1991
10076,British Virgin Islands,23161.0,2001
5090,Guernsey,59807.0,2001
12616,New Caledonia,245580.0,2009
15717,Central African Republic,4349921.0,2010
15206,Togo,6191155.0,2010
15739,Equatorial Guinea,696167.0,2010
15729,Chad,11720781.0,2010
15485,Burundi,8444784.0,2010
15675,Cameroon,2865795.0,2010


In [148]:
#now we need a table of the ethnic groups and population percentages for each, so traverse the XML for these elements
eth_pct = []
for element in document.iterfind('country'):
    country_name = element.find('name').text
    for eth in element.getiterator('ethnicgroup'):
        row={}
        row['country'] = country_name
        row['pct'] = float(eth.attrib['percentage'])
        row['groupname'] = eth.text
        eth_pct.append(row)
        
eth_table = pd.DataFrame(eth_pct)
eth_table

,country,groupname,pct
0,Albania,Albanian,95.00
1,Albania,Greek,3.00
2,Greece,Greek,93.00
3,Macedonia,Macedonian,64.20
4,Macedonia,Albanian,25.20
5,Macedonia,Turkish,3.90
6,Macedonia,Gypsy,2.70
7,Macedonia,Serb,1.80
8,Serbia,Serb,82.90
9,Serbia,Montenegrin,0.90


In [149]:
#merge the total population data with the ethnic group percentages
eth_table = eth_table.merge(cur_pop, on='country')
eth_table

,country,groupname,pct,population,year
0,Albania,Albanian,95.00,2800138.0,2011
1,Albania,Greek,3.00,2800138.0,2011
2,Greece,Greek,93.00,233.0,2014
3,Macedonia,Macedonian,64.20,2059794.0,2011
4,Macedonia,Albanian,25.20,2059794.0,2011
5,Macedonia,Turkish,3.90,2059794.0,2011
6,Macedonia,Gypsy,2.70,2059794.0,2011
7,Macedonia,Serb,1.80,2059794.0,2011
8,Serbia,Serb,82.90,7120666.0,2011
9,Serbia,Montenegrin,0.90,7120666.0,2011


In [150]:
#calculate the population of each ethnic group in each country
eth_table['eth_pop'] = (eth_table['pct']/100)*eth_table['population']
eth_table

,country,groupname,pct,population,year,eth_pop
0,Albania,Albanian,95.00,2800138.0,2011,2.660131e+06
1,Albania,Greek,3.00,2800138.0,2011,8.400414e+04
2,Greece,Greek,93.00,233.0,2014,2.166900e+02
3,Macedonia,Macedonian,64.20,2059794.0,2011,1.322388e+06
4,Macedonia,Albanian,25.20,2059794.0,2011,5.190681e+05
5,Macedonia,Turkish,3.90,2059794.0,2011,8.033197e+04
6,Macedonia,Gypsy,2.70,2059794.0,2011,5.561444e+04
7,Macedonia,Serb,1.80,2059794.0,2011,3.707629e+04
8,Serbia,Serb,82.90,7120666.0,2011,5.903032e+06
9,Serbia,Montenegrin,0.90,7120666.0,2011,6.408599e+04


In [152]:
#sum up populations for each ethnic group across all countries and return the top 10
eth_table.groupby('groupname')['eth_pop'].sum().sort_values(ascending=False).head(10)

groupname
Malay              9.002171e+07
Eastern Hamitic    8.283038e+07
Viet/Kinh          7.607838e+07
African            6.214604e+07
Thai               5.108416e+07
Arab-Berber        4.933220e+07
Arab               4.429985e+07
Mestizo            4.079181e+07
Mangbetu-Azande    2.798602e+07
Chinese            2.247096e+07
Name: eth_pop, dtype: float64

In [175]:
#presumably we want the country name for each river, lake and airport, so create a country code lookup table
cty_codes=[]
for element in document.iterfind('country'):
    row={}
    row['country_name'] = element.find('name').text
    row['country_code'] = element.attrib['car_code']
    cty_codes.append(row)

cty_codes = pd.DataFrame(cty_codes)
cty_codes

,country_code,country_name
0,AL,Albania
1,GR,Greece
2,MK,Macedonia
3,SRB,Serbia
4,MNE,Montenegro
5,KOS,Kosovo
6,AND,Andorra
7,F,France
8,E,Spain
9,A,Austria


In [194]:
#iterate through the tree and grab river name, length and country (I'm using the source country, but there is other data
#for countries the river runs through that could be used depending on requirements)
river = []
for element in document.iterfind('river'): 
    try:
        row={}
        row['country_code'] = element.find('source').attrib['country']
        row['name'] = element.find('name').text
        row['length'] = float(element.find('length').text)
        river.append(row)
        #skipping rivers missing any of the required data elements
    except AttributeError:
        pass

pd.DataFrame(river).sort_values('length',ascending=False).head(10).merge(cty_codes, on='country_code')

,country_code,length,name,country_name
0,PE,6448.0,Amazonas,Peru
1,CN,6380.0,Jangtse,China
2,CN,4845.0,Hwangho,China
3,CN,4350.0,Mekong,China
4,R,4400.0,Lena,Russia
5,R,4092.0,Jenissej,Russia
6,ZRE,4374.0,Zaire,Zaire
7,KAZ,4248.0,Irtysch,Kazakhstan
8,RG,4184.0,Niger,Guinea
9,USA,4130.0,Missouri,United States


In [195]:
lake = []
for element in document.iterfind('lake'): 
    try:
        row={}
        row['country_code'] = element.find('located').attrib['country']
        row['name'] = element.find('name').text
        row['area'] = float(element.find('area').text)
        lake.append(row)
    except AttributeError:
        pass

pd.DataFrame(lake).sort_values('area', ascending=False).head(10).merge(cty_codes, on='country_code')

,area,country_code,name,country_name
0,386400.0,R,Caspian Sea,Russia
1,31492.0,R,Ozero Baikal,Russia
2,82103.0,CDN,Lake Superior,Canada
3,59600.0,CDN,Lake Huron,Canada
4,31792.0,CDN,Great Bear Lake,Canada
5,68870.0,EAT,Lake Victoria,Tanzania
6,57800.0,USA,Lake Michigan,United States
7,41650.0,IL,Dead Sea,Israel
8,32893.0,ZRE,Lake Tanganjika,Zaire
9,29600.0,MOC,Lake Malawi,Mozambique


In [198]:
airport = []
for element in document.iterfind('airport'): 
    try:
        row={}
        row['country_code'] = element.attrib['country']
        row['name'] = element.find('name').text
        row['elev'] = int(element.find('elevation').text)
        airport.append(row)
    except:
        pass

pd.DataFrame(airport).sort_values('elev',ascending=False).head(10).merge(cty_codes, on='country_code')

,country_code,elev,name,country_name
0,BOL,4063,El Alto Intl,Bolivia
1,BOL,2905,Juana Azurduy De Padilla,Bolivia
2,CN,4005,Lhasa-Gonggar,China
3,CN,3963,Yushu Batang,China
4,PE,3827,Juliaca,Peru
5,PE,3311,Teniente Alejandro Velasco Astete Intl,Peru
6,PE,2719,Coronel Fap Alfredo Mendivil Duarte,Peru
7,PE,2677,Mayor General FAP Armando Revoredo Iglesias Ai...,Peru
8,EC,2813,Mariscal Sucre Intl,Ecuador
9,MEX,2581,Licenciado Adolfo Lopez Mateos Intl,Mexico
